### Imports

In [2]:
import numpy as np
import pandas as pd
import re
import warnings
import datetime
warnings.filterwarnings("ignore")
import pickle
import scipy
from scipy import signal

In [1]:
#path to functions.ipynb must be specified if not in same directory
%run functions.ipynb

### Opening Dataframe

In [2]:
#reads in the optical flow dataframe
of_path = '/Users/graciezhang/Documents/Hengen-Lab/Hengen-Lab-Work/Notebooks/Files/optical-flow-flickers-df-feather'
df = pd.read_feather(of_path)

In [3]:
#gets all the video names from the dataframe
video_names = df["video_filename"].unique()

<b>optical_flow</b>: optical flow dataframe

In [4]:
#pickles the optical flow dataframe
filename = 'optical_flow_df'
outfile = open(filename,'wb')
pickle.dump(df, outfile)
outfile.close()

### pickling dataframe w/ added columns

In [5]:
#median filter on normalized OF
def median_filter(video_name, filter_length = 5):
    ref_df = df[df['video_filename'] == video_name]
    return scipy.signal.medfilt(ref_df['percentile_normalized_flow'].values, filter_length)

In [6]:
#is in high activity
def is_in_hact(threshold_value, median_filter):
    is_in = []
    for i in range(len(median_filter)):
        if median_filter[i] >= threshold_value:
            is_in.append(True) 
        else:
            is_in.append(False)
    return is_in

Procedure:

1. Apply Median Filter to Normalized Flow Values, width being 5
2. If NFV > threshold, threshold being 0.35, annotation = 1

In [7]:
#consistent with the indexes of the dataframe
med_fil = []
for i, video_name in enumerate(video_names):
    med_fil.extend(median_filter(video_name))

Calculated short bands/traces based on a 0.35 threshold:

In [8]:
shortb = is_in_hact(0.35, med_fil)
pred = np.asarray(shortb).astype(int)
normalized = df['percentile_normalized_flow']
videos = df['video_filename']
flickers = df['flicker_flavor']

In [9]:
trace_df = df.copy()
trace_df['median_filter_pnf'] = med_fil
trace_df['short_band_set_threshold'] = shortb
trace_df['traces'] = pred

<b>traces</b>: optical flow dataframe with trace information included

In [10]:
filename = 'traces_df'
outfile = open(filename,'wb')
pickle.dump(trace_df, outfile)
outfile.close()

<b>mod_df</b>: optical flow dataframe with only relevant information

In [11]:
mod_df = trace_df[['dataset_name', 'video_filename', 'flicker_flavor', 'percentile_normalized_flow', 'median_filter_pnf', 'traces']].copy()

In [7]:
filename = 'mod_df'
outfile = open(filename,'wb')
pickle.dump(mod_df, outfile)
outfile.close()

<b>mouse_vids</b>: dictionary of all videos + the mouse they belong to

In [35]:
mouse_vids = {}
for video_name in video_names:
    mouse_name = mod_df.loc[mod_df["video_filename"] == video_name]["dataset_name"].unique()[0]
    mouse_vids[video_name] = mouse_name

In [36]:
filename = 'mouse_vids'
outfile = open(filename,'wb')
pickle.dump(mouse_vids, outfile)
outfile.close()

### out of frame

In [10]:
oof106_df = pd.read_csv("out_of_frame - CAF106.csv")

In [11]:
oof106_df

,CAF00106-20210607T110838-120839.mp4,CAF00106-20210607T120839-130839.mp4,CAF00106-20210607T130839-140839.mp4,CAF00106-20210607T140839-150840.mp4,CAF00106-20210607T150840-160841.mp4,CAF00106-20210607T160841-170841.mp4,CAF00106-20210607T170841-180842.mp4,CAF00106-20210607T180842-190842.mp4,CAF00106-20210607T190842-200843.mp4,CAF00106-20210607T200843-210843.mp4,...,CAF00106-20210608T010845-020846.mp4,CAF00106-20210608T020846-030847.mp4,CAF00106-20210608T030847-040848.mp4,CAF00106-20210608T040848-050849.mp4,CAF00106-20210608T050849-060849.mp4,CAF00106-20210608T060849-070850.mp4,CAF00106-20210608T070850-080850.mp4,CAF00106-20210608T080850-090851.mp4,CAF00106-20210608T090851-100851.mp4,CAF00106-20210608T100851-110522.mp4
0,3:56,26:48:00,0:00:00,0:20:00,0:00:00,0:00:00,0:00:00,0:00:00,0:00:00,0:00:00,...,10:58:00,0:00:00,0:00:00,0:00:00,0:00:00,0:00:00,0:00:00,0:00:00,0:00:00,0:00:00
1,4:02,27:07:00,52:26:00,0:51:00,1:00:00,58:46:00,2:33:00,1:00:00,1:00:00,1:00:00,...,35:25:00,27:03:00,1:00:00,48:39:00,4:59:00,1:00:00,45:52:00,1:00:00,31:10:00,1:00:00
2,5:04,44:50:00,53:05:00,3:16:00,NaN,59:29:00,6:46:00,NaN,NaN,NaN,...,35:36:00,35:48:00,NaN,49:34:00,6:35:00,NaN,49:36:00,NaN,31:20:00,NaN
3,5:10,45:22:00,59:50:00,3:39,NaN,1:00:00,14:02:00,NaN,NaN,NaN,...,38:37:00,36:35:00,NaN,1:00:01,10:47:00,NaN,1:00:00,NaN,45:01:00,NaN
4,18:50:00,45:35:00,59:54:00,11:59:00,NaN,NaN,14:53:00,NaN,NaN,NaN,...,43:13:00,37:10:00,NaN,NaN,11:01:00,NaN,NaN,NaN,45:47:00,NaN
5,19:06:00,45:44:00,59:56:00,12:12:00,NaN,NaN,15:56:00,NaN,NaN,NaN,...,44:47:00,43:49:00,NaN,NaN,1:00:00,NaN,NaN,NaN,46:11:00,NaN
6,20:00,46:23:00,NaN,12:27,NaN,NaN,23:09:00,NaN,NaN,NaN,...,46:56:00,43:59:00,NaN,NaN,NaN,NaN,NaN,NaN,46:39:00,NaN
7,20:17,58:55:00,NaN,12:28,NaN,NaN,1:00:00,NaN,NaN,NaN,...,48:20:00,44:20:00,NaN,NaN,NaN,NaN,NaN,NaN,1:00:00,NaN
8,20:28,59:00:00,NaN,14:04:00,NaN,NaN,NaN,NaN,NaN,NaN,...,48:34:00,45:07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,20:35,1:00:00,NaN,14:10:00,NaN,NaN,NaN,NaN,NaN,NaN,...,50:07:00,53:07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#gets all the caf106 videos
oof106_vids = list(oof106_df)

In [14]:
oof106_frames = {}
for vid in oof106_vids:
    time = datetime.strptime()
    #converts to seconds
    frames = oof106_df[vid].values / 15
    oof106_frames[vid] = frames

TypeError: unsupported operand type(s) for /: 'str' and 'int'

### readable arrays

In [17]:
#gets the total number of frames for each video
total_frames = mod_df['video_filename'].value_counts()

In [15]:
#stores the intervals of flickers
all_flickers = {}

In [22]:
#stores the intervals of traces
all_traces = {}

In [23]:
#stores the timestamps of the traces for easier comparison between the videos and the generated intervals
timestamps = {}

In [21]:
for video_name in video_names:
    #isolates video
    video_df = mod_df.loc[mod_df['video_filename'] == video_name]
    
    #find consecutive flicker interval
    flicker = video_df['flicker_flavor'].values
    groups = find_groups(flicker).get('WAKE_WITHIN_NREM')
    
    vid_flickers = []
    if groups != None:
        #iterates through each group
        for group in groups:
            if calculate_frames(group[0], group[1], 450) != None:
                vid_flickers.append(group)
        all_flickers[video_name] = vid_flickers
    else:
        all_flickers[video_name] = None

In [24]:
for i, video_name in enumerate(video_names):
    video_df = mod_df.loc[mod_df['video_filename'] == video_name]
    cur_flickers = all_flickers.get(video_name)
    if cur_flickers != None:
        for flicker in cur_flickers:
            name, ants = annotations(video_name, video_df, flicker, 450, 'traces')
            all_traces[name] = ants
    else:
        all_traces[video_name] = None

In [25]:
for key in all_traces.keys():
    if all_traces[key] is None:
        timestamps[key] = None
    else:
        groups = find_groups(all_traces[key]).get(1)
        if groups != None:
            all_ants = []
            for j, sub_ant in enumerate(groups):
                start = (sub_ant[0])/15
                end = (sub_ant[1])/15

                start_time = str(datetime.timedelta(seconds = start))
                end_time = str(datetime.timedelta(seconds = end))
                
                all_ants.append([start_time, end_time])
                
            timestamps[key] = all_ants
        else:
            timestamps[key] = None

In [72]:
all_flickers

{'e3v819c-20200807T1404-1509.mp4': [(7575, 7604)],
 'e3v819c-20200807T1509-1609.mp4': None,
 'e3v819c-20200807T1609-1709.mp4': [(48975, 49018),
  (50449, 50463),
  (50480, 50487),
  (52883, 52906)],
 'e3v819c-20200807T1709-1809.mp4': [(14510, 14517)],
 'e3v819c-20200807T1809-1909.mp4': [(9979, 9986)],
 'e3v819c-20200807T1909-2009.mp4': [(14830, 14895),
  (25842, 25857),
  (25873, 25910),
  (26677, 26710),
  (30790, 30802)],
 'e3v819c-20200807T2009-2109.mp4': [(47698, 47728),
  (48090, 48108),
  (48110, 48120),
  (53044, 53119),
  (53123, 53172)],
 'e3v819c-20200807T2109-2209.mp4': [(31544, 31597),
  (31677, 31749),
  (35044, 35057),
  (37164, 37190)],
 'e3v819c-20200807T2209-2309.mp4': [(4130, 4202),
  (4482, 4564),
  (6499, 6547),
  (12116, 12180)],
 'e3v819c-20200807T2309-0009.mp4': None,
 'e3v819c-20200808T0009-0109.mp4': None,
 'e3v819c-20200808T0109-0209.mp4': None,
 'e3v819c-20200808T0209-0309.mp4': [(45510, 45515), (45542, 45569)],
 'e3v819c-20200808T0309-0409.mp4': [(30718, 307

In [71]:
all_traces

{'e3v819c-20200807T1404-1509_f7575-f7604.mp4': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [68]:
timestamps

{'e3v819c-20200807T1404-1509_f7575-f7604.mp4': [['0:00:27.800000',
   '0:00:29.733333'],
  ['0:00:31.200000', '0:00:41.800000']],
 'e3v819c-20200807T1509-1609.mp4': None,
 'e3v819c-20200807T1609-1709_f48975-f49018.mp4': [['0:00:00',
   '0:00:04.466667'],
  ['0:00:05.600000', '0:00:06.800000'],
  ['0:00:35.866667', '0:00:37.666667']],
 'e3v819c-20200807T1609-1709_f50449-f50463.mp4': [['0:00:21.133333',
   '0:00:26.133333'],
  ['0:00:36', '0:00:39.866667']],
 'e3v819c-20200807T1609-1709_f50480-f50487.mp4': [['0:00:19.333333',
   '0:00:24.333333'],
  ['0:00:34.200000', '0:00:38.066667']],
 'e3v819c-20200807T1609-1709_f52883-f52906.mp4': [['0:00:36.133333',
   '0:00:39.466667']],
 'e3v819c-20200807T1709-1809_f14510-f14517.mp4': [['0:00:36',
   '0:00:37.600000'],
  ['0:00:43', '0:00:45.466667']],
 'e3v819c-20200807T1809-1909_f9979-f9986.mp4': None,
 'e3v819c-20200807T1909-2009_f14830-f14895.mp4': [['0:00:36.733333',
   '0:00:40']],
 'e3v819c-20200807T1909-2009_f25842-f25857.mp4': None,
 'e3

Pickles all readable arrays.

In [30]:
filename = 'wn_flickers'
outfile = open(filename,'wb')
pickle.dump(all_flickers, outfile)
outfile.close()

In [31]:
filename = 'wn_traces'
outfile = open(filename,'wb')
pickle.dump(all_traces, outfile)
outfile.close()

In [32]:
filename = 'wn_timestamps'
outfile = open(filename,'wb')
pickle.dump(timestamps, outfile)
outfile.close()